In [ ]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from data_loading import load_dataset

X, y = load_dataset()
# Preprocessing
y_categorical = to_categorical(y)  # Convert integers to one-hot encoded labels

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [ ]:
# Transformer
from scipy import signal

# Define the original and target sampling frequencies
original_fs = 704.0  # Original sampling frequency (Hz)
target_fs = 128.0  # Target sampling frequency (Hz)

# Calculate the resampling factor
resampling_factor = target_fs / original_fs

# Resample the ECG signals in X_train
X_train_resampled = []
for entry in X_train:
    resampled_signal = signal.resample(entry, int(len(entry) * resampling_factor))
    X_train_resampled.append(resampled_signal)

# Resample the ECG signals in X_test
X_test_resampled = []
for entry in X_test:
    resampled_signal = signal.resample(entry, int(len(entry) * resampling_factor))
    X_test_resampled.append(resampled_signal)

# Convert the resampled data to NumPy arrays
X_train_resampled = np.array(X_train_resampled)
X_test_resampled = np.array(X_test_resampled)
# Quantization of ECG data
num_bins = 50  # Define the number of bins you'd like to use

# Initialize KBinsDiscretizer
kbin = KBinsDiscretizer(n_bins=num_bins, encode='ordinal', strategy='uniform')
kbin.fit(np.concatenate((X_test_resampled, X_test_resampled)))

X_train_tokenized = kbin.transform(X_train_resampled)
X_test_tokenized = kbin.transform(X_test_resampled)
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset

# Initialize a pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)


# Define a custom classifier model
class ECGClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ECGClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, num_classes)  # BERT hidden size is 768

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits


# Convert tokenized data to PyTorch tensors
X_train_tokenized = torch.LongTensor(X_train_tokenized)
X_test_tokenized = torch.LongTensor(X_test_tokenized)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# Create DataLoader for training and testing data
train_data = TensorDataset(X_train_tokenized, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = TensorDataset(X_test_tokenized, y_test)
test_loader = DataLoader(test_data, batch_size=32)

# Initialize the model
model = ECGClassifier(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, labels = batch
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask=(input_ids != 0))  # Attention mask for BERT
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, labels = batch
        logits = model(input_ids, attention_mask=(input_ids != 0))
        predicted = torch.argmax(logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.2%}")
